In [1]:
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

In [ ]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features = np.concatenate([
        np.mean(mfccs, axis=1),
        np.mean(chroma, axis=1),
        np.mean(spectral_centroid, axis=1),
        np.mean(spectral_bandwidth, axis=1)
    ])
    return features

In [ ]:
def load_dataset(data_dir):
    features, labels = [], []
    for label, gender in enumerate(['male', 'female']):
        gender_dir = os.path.join(data_dir, gender)
        for file_name in os.listdir(gender_dir):
            file_path = os.path.join(gender_dir, file_name)
            try:
                audio_features = extract_features(file_path)
                features.append(audio_features)
                labels.append(label)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")
    
    return np.array(features), np.array(labels)

In [ ]:
data_dir = 'audio_dataset'
features, labels = load_dataset(data_dir)
print(features[0],labels[0])

print(f"Total samples in dataset: {len(labels)}")
print(f"Number of male samples: {np.sum(labels == 0)}")
print(f"Number of female samples: {np.sum(labels == 1)}")

scaler = StandardScaler()
features = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.8, random_state=42)

print(f"Training samples: {len(y_train)}")
print(f"Testing samples: {len(y_test)}")

model = SVC()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Total samples in dataset: 1739
Number of male samples: 739
Number of female samples: 1000
Training samples: 1391
Testing samples: 348
Accuracy: 100.00%


In [45]:
print(features[0],labels[0])


[-228.50111     83.37447     13.948432    37.37506      8.219474
   -5.779841   -21.11218     -7.528347   -19.253016    12.824012
  -20.15146      3.5419867   -7.018186 ] 0


In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, target_names=['Male', 'Female'])

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


Confusion Matrix:
[[156   0]
 [  0 192]]

Classification Report:
              precision    recall  f1-score   support

        Male       1.00      1.00      1.00       156
      Female       1.00      1.00      1.00       192

    accuracy                           1.00       348
   macro avg       1.00      1.00      1.00       348
weighted avg       1.00      1.00      1.00       348



In [ ]:
import joblib
joblib.dump(model, 'gender_classifier.pkl')
print("Model saved to 'gender_classifier.pkl'")


Model saved to 'gender_classifier.pkl'


In [ ]:
model = joblib.load('gender_classifier.pkl')
print("Model loaded successfully")


Model loaded successfully


In [47]:
def predict_gender(file_path):
    audio_features = extract_features(file_path)
    audio_features = scaler.transform([audio_features])  # Normalize
    prediction = model.predict(audio_features)
    return "Male" if prediction[0] == 0 else "Female"

test_file = 'ex.mp3'
print(f"Predicted Gender: {predict_gender(test_file)}")


Predicted Gender: Male
